In [2]:
%cd ..
%cd ..
%cd ..
%cd ..

/Users/nad/hse/2023-24/spring_proj2/res/RESO/generators/neural/transformer
/Users/nad/hse/2023-24/spring_proj2/res/RESO/generators/neural
/Users/nad/hse/2023-24/spring_proj2/res/RESO/generators
/Users/nad/hse/2023-24/spring_proj2/res/RESO


/Users/nad/opt/anaconda3/envs/reso02/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


#### Load the model and tokenizer

In [11]:
from transformers import AutoTokenizer, GPT2LMHeadModel

model_name_or_path = 'generators/neural/transformer/gpt2/checkpoints/EPOCH260'

model = GPT2LMHeadModel.from_pretrained(model_name_or_path, device_map='cpu')
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, device_map='cpu')

In [ ]:
# !pip install note_seq

#### Funcs to decode from tokens

In [44]:
import note_seq

NOTE_LENGTH_16TH_120BPM = 0.25 * 60 / 120
BAR_LENGTH_120BPM = 4.0 * 60 / 120


def empty_note_sequence(qpm=120.0, total_time=0.0):
    note_sequence = note_seq.protobuf.music_pb2.NoteSequence()
    note_sequence.tempos.add().qpm = qpm
    note_sequence.ticks_per_quarter = note_seq.constants.STANDARD_PPQ
    note_sequence.total_time = total_time
    return note_sequence


def token_sequence_to_note_sequence(token_sequence):

    if isinstance(token_sequence, str):
        token_sequence = token_sequence.split()

    note_sequence = empty_note_sequence()

    track_count = 0
    current_time = 0
    for token_index, token in enumerate(token_sequence):
        # print("current time:", current_time)
        if token == "PIECE_START":
            pass
        elif token == "PIECE_END":
            print("The end.")
            break
        elif token == "TRACK_START":
            current_bar_index = 0
            track_count += 1
            pass
        elif token == "TRACK_END":
            pass
        elif token == "KEYS_START":
            pass
        elif token == "KEYS_END":
            pass
        elif token.startswith("KEY="):
            pass
        elif token == "BAR_START":
            current_time = current_bar_index * BAR_LENGTH_120BPM
            current_notes = {}
        elif token == "BAR_END":
            current_bar_index += 1
            pass
        elif token.startswith("NOTE_ON"):
            pitch = int(token.split("=")[-1])
            note = note_sequence.notes.add()
            note.start_time = current_time
            note.end_time = current_time + 4 * NOTE_LENGTH_16TH_120BPM
            note.pitch = pitch
            note.instrument = 0
            note.program = 0
            note.velocity = 80
            current_notes[pitch] = note
            # print(note_sequence)
        elif token.startswith("NOTE_OFF"):
            pitch = int(token.split("=")[-1])
            if pitch in current_notes:
                note = current_notes[pitch]
                note.end_time = current_time
        elif token.startswith("TIME_DELTA"):
            delta = float(token.split("=")[-1]) * NOTE_LENGTH_16TH_120BPM
            current_time += delta
            # print("delta: ", delta)
        elif token.startswith("DENSITY="):
            pass
        elif token == "[PAD]":
            pass
        else:
            #print(f"Ignored token {token}.")
            pass

    return note_sequence

Add the model to the GPU, if available

In [14]:
import torch 
if torch.cuda.is_available():
    device = torch.device("cuda")
    model.to(device)

In [51]:
input_ids = tokenizer.encode("PIECE_START GENRE=OTHER", return_tensors="pt")

number_of_regenerations = 4
for i in range(number_of_regenerations):
    generated_ids = model.generate(
        input_ids,
        max_length=2048,
        do_sample=True,
        temperature=0.75,
        eos_token_id=tokenizer.encode("TRACK_END")[0]
    )
    input_ids = generated_ids
token_sequence = tokenizer.decode(input_ids[0])
note_sequence = token_sequence_to_note_sequence(token_sequence)

# In case predicted TIME_DELTAs for notes are strange

# def set_start_times(note_sequence, ticks_per_quarter):
#     qpm = note_sequence.tempos[0].qpm
#     seconds_per_quarter = 60.0 / qpm
#     duration_ticks = ticks_per_quarter
#     duration_seconds = seconds_per_quarter * (duration_ticks / ticks_per_quarter)
#     duration_seconds = 0.2
#     current_time = 0.0
#     for note in note_sequence.notes:
#         note.start_time = current_time
#         note.end_time = current_time + duration_seconds
#         current_time += duration_seconds
# set_start_times(note_sequence, note_sequence.ticks_per_quarter)

print(note_sequence)
midi_file_path = 'output.mid'
note_seq.sequence_proto_to_midi_file(note_sequence, midi_file_path)

ticks_per_quarter: 220
tempos {
  qpm: 120
}
notes {
  pitch: 68
  velocity: 80
  start_time: 0.10546875
  end_time: 0.37890625
}
notes {
  pitch: 70
  velocity: 80
  start_time: 0.38020833333333331
  end_time: 0.66015625
}
notes {
  pitch: 68
  velocity: 80
  start_time: 0.38671875
  end_time: 0.83984375
}
notes {
  pitch: 71
  velocity: 80
  start_time: 0.62109375
  end_time: 0.859375
}
notes {
  pitch: 73
  velocity: 80
  start_time: 0.90364583333333337
  end_time: 1.5
}
notes {
  pitch: 64
  velocity: 80
  start_time: 1.1432291666666667
  end_time: 1.4895833333333333
}
notes {
  pitch: 63
  velocity: 80
  start_time: 1.40625
  end_time: 1.6940104166666667
}
notes {
  pitch: 61
  velocity: 80
  start_time: 1.6510416666666667
  end_time: 1.9973958333333335
}
notes {
  pitch: 63
  velocity: 80
  start_time: 1.9479166666666667
  end_time: 2
}
notes {
  pitch: 63
  velocity: 80
  start_time: 2
  end_time: 3.3294270833333335
}
notes {
  pitch: 66
  velocity: 80
  start_time: 2.2239583333